In [2]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import re
import datetime
from tqdm import tqdm_notebook

### 네이버 영화-> 현재 상영영화에서 평점순 top 10 영화의 정보가 담긴 url을 수집한다

In [3]:
def movie_crawler(date):
    url = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn'
    params = {'sel':'cur',
             'tg':0,
             'date':date}

    resp = requests.get(url,params=params)
    soup = BeautifulSoup(resp.text)
    table=soup.find('table',class_='list_ranking')
    top10_movie=table.find_all('tr')[2:12]
    day_top10_link=[]
    base_url='https://movie.naver.com'
    for movie in top10_movie:

        href=movie.find('a').get('href')
        movie_url=base_url+href
        day_top10_link.append(movie_url)
    return day_top10_link

### URL의 xml을 뷰티풀숩으로 파싱해 영화관련 정보를 데이터프레임 형식으로 저장한다 

In [4]:
def get_movie_info(date,url_list):
    movie_info = {
        '제목':[],
        '날짜':[],
        '평점':[],
        '장르':[],
        '국가':[],
        '상영시간':[],
        '개봉일':[],
        '줄거리':[]            
        }    
    for url in url_list:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text)
        # 제목 추출
        title = soup.find('h3',class_='h_movie')
        title = title.find('a').text
        # 평점 추출
        score=soup.find('div',class_='main_score')
        score=score.find('div',class_='star_score').text.split()[2][:5]
        
        infos=soup.find('dl',class_='info_spec')
        info_list = infos.select("dd p span")
        # 국가 추출
        country=info_list[1].text
        country = re.sub('\s','',country)
        # 장르 추출
        genre = info_list[0].text
        genre = re.sub('\s','',genre)
        # 상영시간 추출
        running_time = info_list[2].text
        # 개봉일 추출
        opening_days = info_list[3].text
        opening_days = re.sub('\s','',opening_days)
        # 줄거리 추출
        story=soup.find('div',class_='obj_section')
        story=story.find('p').text
        story = re.sub('\r\xa0', "", story)
        movie_info['제목'].append(title)
        movie_info['날짜'].append(date)
        movie_info['평점'].append(score)
        movie_info['장르'].append(genre)
        movie_info['국가'].append(country)
        movie_info['상영시간'].append(running_time)
        movie_info['개봉일'].append(opening_days)
        movie_info['줄거리'].append(story)
    result=pd.DataFrame(movie_info)
    return result

### 위의 두 함수를 활용해 설정한 기간 내의 영화정보를 데이터 프레임으로 저장한다

In [5]:
def append_movie_info(start_date, end_date):
    total_info = pd.DataFrame({
        '제목':[],
        '날짜':[],
        '평점':[],
        '장르':[],
        '국가':[],
        '상영시간':[],
        '개봉일':[],
        '줄거리':[]            
        })

    date_list = pd.date_range(start=start_date, end = end_date)
    
    for date in tqdm_notebook(date_list):
        url_list = movie_crawler(date)
        daily_movie_info=get_movie_info(date,url_list)
        total_info=pd.concat([total_info,daily_movie_info],ignore_index=True)
    return total_info

### 오늘의 영화정보를 크롤링해 과거 데이터에 추가한다

In [6]:
data = append_movie_info('20200501',datetime.date.today())
data.to_csv('movie_crawling.csv',encoding='utf-8', index=False)
print('=====END Movie Crawling=====')

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))



=====END Movie Crawling=====


In [13]:
data['날짜'] = data['날짜'].astype(str)

In [14]:
data

,제목,날짜,평점,장르,국가,상영시간,개봉일,줄거리
0,위대한 쇼맨,2020-05-01,9.30점,"드라마,뮤지컬",미국,104분,"2020.05.21재개봉,2017.12.20개봉","쇼 비즈니스의 창시자이자, 꿈의 무대로 전세계를 매료시킨 남자‘바넘’의 이야기에서 ..."
1,에이리언 2,2020-05-01,없음,"SF,공포,스릴러,액션",미국,137분,1986.12.24개봉,에이리언과의 싸움에서 간신히 살아남은 리플리는 잠든 채로 57년을 떠돌다 구조선에 ...
2,패왕별희 디 오리지널,2020-05-01,9.64점,드라마,"중국,홍콩",171분,"2020.05.01재개봉,2017.03.30재개봉,1993.12.24개봉",어렸을 때부터 함께 경극을 해온 ‘두지’(장국영)와 ‘시투’(장풍의). 세상에 둘...
3,인생,2020-05-01,5.00점,드라마,"중국,대만",125분,1995.05.27개봉,"1940년대 중국, 부유한 지주집 아들인 부귀(갈우 분)는 아름다운 아내(공리 분)..."
4,우리들,2020-05-01,9.21점,드라마,한국,94분,2016.06.16개봉,"그 여름, 나에게도 친구가 생겼다… “내 마음이 들리니” 언제나 혼자인 외톨이 선..."
...,...,...,...,...,...,...,...,...
545,톰보이,2020-06-24,8.85점,드라마,프랑스,82분,2020.05.14개봉,"새로 이사 온 아이, ‘미카엘’.파란색을 좋아하고, 끝내주는 축구 실력과 유난히 잘..."
546,러빙 빈센트,2020-06-24,9.18점,"애니메이션,미스터리","영국,폴란드",107분,"2018.12.13재개봉,2017.11.09개봉",살아생전 단 한 점의 그림만을 팔았던 화가 ‘빈센트’의 죽음 후 1년. 아버지의 부...
547,어느 가족,2020-06-24,9.25점,드라마,일본,121분,2018.07.26개봉,할머니의 연금과 물건을 훔쳐 생활하며 가난하지만 웃음이 끊이지 않는 어느 가족.우연...
548,시간을 달리는 소녀,2020-06-24,9.43점,"애니메이션,가족,드라마,판타지",일본,97분,"2016.01.14재개봉,2014.04.25재개봉,2007.06.14개봉",저 마코토에게는 남들에게 말 못할 비밀이 하나 있어요. 타임리프라고 하는 능력이죠....


In [6]:
# def append_today():
#     prev_data = pd.read_csv('movie_crawling.csv',index_col=0)
#     now=datetime.datetime.today()
#     date = "{}-{}-{}".format(now.year, now.month, now.day)
#     url_list = movie_crawler(date)
#     daily_movie_info=get_movie_info(date,url_list)
#     new_data=pd.concat([prev_data,daily_movie_info],ignore_index=True)
#     return new_data

In [ ]:
# data.to_csv('movie_crawling.csv')

In [ ]:
# data = append_today()